In [1]:
from pathlib import Path

In [5]:
CWD = Path(".").resolve().parent/"code_sample"
CWD

PosixPath('/home/pyuser/workspace/code_sample')

In [46]:
files = tuple(filter(
    lambda x: not x.match("*/.ipynb_checkpoints/*"),
    CWD.glob("**/*.py")
))

files

(PosixPath('/home/pyuser/workspace/code_sample/python/module1.py'),
 PosixPath('/home/pyuser/workspace/code_sample/python/module2.py'),
 PosixPath('/home/pyuser/workspace/code_sample/python/submodule/submodule1.py'))

In [40]:
test_file = [files[1]]

import re

from codetags.utils import select_tags

tags = select_tags()
regex = re.compile(f'(?:(?:# )({tags})(?::))(.*?)(?:(?:<(.*?)>)|(?:\n))')

target = []
for file in test_file:
    with file.open(mode='r') as f:
        print(f.readline())
        print(f.readline())
        print(f.readline())

# target

"""

This is a sample module for developing/testing codetags

"""



In [54]:
RE_P = re.compile(r"p:([1-9]{1})")
RE_D = re.compile(
    r"d:(((?:19|20)[0-9][0-9])-(0?[1-9]|1[012])-(0[1-9]|[12][0-9]|3[01]))"
)

def parse_fields(tag_field):
    """ Parse/extract the 'p'riority and 'd'ate value from the tag field inside
    the brackets '<>'. Can have both or only one of the tags.
    Expects a string with the p or d tag and respetives values. 'p' should have
    a single integer 1-9 and 'd' should return a date in the format yyyy-mm-dd.

    Parameters
    ----------
    tag_field: str
    """

    priority = 1
    date = 'yyyy-mm-dd'

    if (tf := tag_field) and tf.strip() != '':
        if (p_match := RE_P.search(tag_field)):
            priority = p_match.group(1)

        if (d_match := RE_D.search(tag_field)):
            date = d_match.group(1)

    return priority, date

In [56]:
test_file = [files[1]]

tags = select_tags()
regex = re.compile(f'(?:(?:# )({tags})(?::))(.*?)(?:(?:<(.*?)>)|(?:\n))')

findings = []
for file in test_file:
    with file.open(mode='r') as f:
        for l_no, line in enumerate(f, start=1):
            if (match := regex.search(line)):
                priority, date = parse_fields(match.group(3))

                findings.append({
                    'path': str(file),
                    'tag': match.group(1),
                    'priority': str(priority),
                    'date': date,
                    'line': l_no,
                    'msg': match.group(2).strip(),
                 })

findings

[{'path': '/home/pyuser/workspace/code_sample/python/module2.py',
  'tag': 'BUG',
  'priority': '1',
  'date': 'yyyy-mm-dd',
  'line': 7,
  'msg': "'bug', module2, line 7, no closing brackets"},
 {'path': '/home/pyuser/workspace/code_sample/python/module2.py',
  'tag': 'FIXME',
  'priority': '1',
  'date': 'yyyy-mm-dd',
  'line': 10,
  'msg': "'fix me', module2, line 10, no closing brackets"},
 {'path': '/home/pyuser/workspace/code_sample/python/module2.py',
  'tag': 'NOTE',
  'priority': '1',
  'date': 'yyyy-mm-dd',
  'line': 13,
  'msg': "'note', module2, line 13, no closing brackets"},
 {'path': '/home/pyuser/workspace/code_sample/python/module2.py',
  'tag': 'TODO',
  'priority': '1',
  'date': 'yyyy-mm-dd',
  'line': 16,
  'msg': "'todo', module2, line 16, no closing brackets"},
 {'path': '/home/pyuser/workspace/code_sample/python/module2.py',
  'tag': 'BUG',
  'priority': '1',
  'date': 'yyyy-mm-dd',
  'line': 19,
  'msg': "'bug', module2, line 19"},
 {'path': '/home/pyuser/works